<div style="text-align: center;">
    <img src=https://d1.awsstatic.com/logos/aws-logo-lockups/poweredbyaws/PB_AWS_logo_RGB_REV_SQ.8c88ac215fe4e441dc42865dd6962ed4f444a90d.png width="350" class="center">
</div>
<center> Welcome to the Scooters Graph Demo! </center>
<center><b> YouTube channel: </b> <a href="https://www.youtube.com/@awsdevelopers">AWS Developers</a></center>

## Introduction
#### Welcome to our tutorial on "Implementing a Graph database for a Scooters Business on AWS".
In this Notebook, we will load the graph datasets we have generated through our Scooters CDK stacks and then use Gremlin queries, to explore this data in Amazon Neptune. Below, you'll also find some magic commands, such as "%graph_notebook_version" or "%status". These are to make sure your environment is ready to run, without any errors. Run them and you should see the metadata about our environment; e.g. cluster name deployed by our CDK stack, cluster status, etc. 

- Note: at the of this notebook, there's an optional step to delete all the data in this Neptune cluster.

In [ ]:
%graph_notebook_version

In [ ]:
%graph_notebook_config

In [ ]:
%status

### Quick operations
**Add two Nodes and 1 edge**

In [ ]:
%%gremlin

g.addV('scooter').property(id, 'scooter_xyz').property('fuel_pct', 26)

In [ ]:
%%gremlin

g.addV('warehouse').property(id, 'warehouse_cambs_south_01').property('location', 'Cambridge South,UK')

In [ ]:
%%gremlin

g.V('scooter_xyz').addE('is_at').to(__.V('warehouse_cambs_south_01')).property('time_gps', 'time_1708699174,52.1774256,0.1454513,').next()

In [ ]:
%%gremlin

g.V('scooter_xyz').outE().inV().path().by(elementMap())

---

In [ ]:
import requests, json, traceback

<b> Requirement: </b><br>
In the next cell, change the parameter values to those from the CDK deployment output or from the AWS console:
1. Open the AWS console, go to CloudFormation and click on our CDK stack called "ScootersNeptuneStack".
2. Click on the second tab, called "Outputs".
3. Copy the value of outputneptuneendpoint and replace this to the value of NEPTUNE_SERVER_ENDPOINT
4. Copy the value of outputneptuneiamrole and replace this to the value of IAM_ROLE_ARN
5. Copy the value of outputs3bucket and replace this to the value of S3_BUCKET
6. For AWS_REGION, enter the AWS region where you're doing all this demo; e.g. eu-west-1


In [ ]:
# Input Neptune parameters, to load the generated data:
NEPTUNE_SERVER_ENDPOINT = 'change_me'
IAM_ROLE_ARN = 'change_me'
S3_BUCKET = 'change_me'
AWS_REGION = 'change_me'

In [ ]:
# Neptune endpoint. You don't need to change anything here, unless you changed the Neptune's database port.
PORT = 8182
ENDPOINT = f"https://{NEPTUNE_SERVER_ENDPOINT}:{PORT}/loader"

### Neptune Loader Command

Loads data from an Amazon S3 bucket into a Neptune DB instance.

To load data, you must send an HTTP POST request to the https://your-neptune-endpoint:port/loader endpoint. The parameters for the loader request can be sent in the POST body or as URL-encoded parameters.

The S3 bucket must be in the same AWS Region as the cluster. In our case, it is as we deployed it using our CDK stack. The same for the required privileges, as the IAM role attached to this cluster has been already granted with S3 access to our CDK-created bucket.

If you want to see all options for the loader, go to: https://docs.aws.amazon.com/neptune/latest/userguide/load-api-reference-load.html

### Load Vertices / Nodes

Let's now load the Nodes dataset, generated by our Lambda function. 


<b> Important:</b>The next cell won't return any information. Just carry on with the next cells

In [ ]:
# Prepare Neptune API input
headers = {'Content-Type': 'application/json'}
data = {
      "source" : f"s3://{S3_BUCKET}/scooters-graph-demo/neptune/data/vertices.csv",
      "format" : "csv",
      "iamRoleArn" : f"{IAM_ROLE_ARN}",
      "region" : f"{AWS_REGION}",
      "failOnError" : "FALSE",
      "parallelism" : "MEDIUM",
      "updateSingleCardinalityProperties" : "TRUE",
      "queueRequest" : "TRUE"
}

# Load CSV file into Neptune
response = requests.post(ENDPOINT, data=json.dumps(data), headers=headers)

### Show response from previous load

We now create a simple function, to request the status from the Neptune Loader API. This will help us in know what happened to our POST command above to load the Vertices.csv file

In [ ]:
def get_load_status(response_load):
    try:
        # Extract Load ID from Amazon Neptune
        neptune_load_id = response_load.json()['payload']['loadId']

        # Parameters definition, to monitor the Load status
        data = {
            "details" : "true",
            "errors" : "true",
            "page" : "1",
            "errorsPerPage": "3"
        }

        # Query the Neptune Loader endpoint, to collect the status
        response = requests.get(f"{ENDPOINT}/{neptune_load_id}", params=data)

        # Return the status. This can be used for polling; i.e. in a Step Function workflow.
        return json.dumps(response.json(), separators=(',', ':'), indent=4)
    except Exception as e:
        print('Error while fetching status: {}'.format(e))
        traceback.print_exc()

Let's now see what the Neptune Loader returns, by querying the Loader endpoint:

<b> Important:</b> read carefully the returned message from the loader. This will tell you if it hasn't started yet (```"status":"LOAD_NOT_STARTED"```), or if it has completed the load (```"status":"LOAD_COMPLETED"```), or if it's still in progress (```"status":"LOAD_IN_PROGRESS"```), or something else (e.g. ERROR). It will also show you how many nodes has loaded, timestamp, and other information. This is very useful, for example when you may be duplicating data by mistake or not creating your CSV data files properly.

Run many times the next cell, until it tells you the load has completed. i.e. If the output is similar to this one, then wait 2 seconds and rerun the ```print(get_load_status(response))``` command, until it says "LOAD_COMPLETED"

```json
{
    "status":"200 OK",
    "payload":{
        "feedCount":[
            {
                "LOAD_IN_PROGRESS":1
            }
        ]
...
```

For more information, see: docs.aws.amazon.com/neptune/latest/userguide/load-api-reference-status-response.html 

In [ ]:
# Monitor Load status
print(get_load_status(response))

### Load Edges
We do the same for our Edges dataset

In [ ]:
# Prepare Neptune API input
headers = {'Content-Type': 'application/json'}
data = {
      "source" : f"s3://{S3_BUCKET}/scooters-graph-demo/neptune/data/edges.csv",
      "format" : "csv",
      "iamRoleArn" : f"{IAM_ROLE_ARN}",
      "region" : f"{AWS_REGION}",
      "failOnError" : "FALSE",
      "parallelism" : "MEDIUM",
      "updateSingleCardinalityProperties" : "TRUE",
      "queueRequest" : "TRUE"
}

# Load CSV file into Neptune
response = requests.post(ENDPOINT, data=json.dumps(data), headers=headers)

In [ ]:
# Monitor Load status. Execute this many times, until the job says LOAD_COMPLETED.
print(get_load_status(response))

### Let's now query the graph data

- Don't forget to see the second and third tab from the results below, as they contain relevant information

In [ ]:
%%gremlin

// This query will return 30 random connected nodes by one hop 
g.V().outE().inV().path().by(elementMap()).limit(30)

In [ ]:
%%gremlin

// This query will return 100 random connected Scooters
g.V().hasLabel('scooter').outE().inV().path().by(elementMap()).limit(100)

In [ ]:
%%gremlin

// This query will return 50 connected Scooters that had an incident, showing the legal case ID related.
g.V().hasLabel('scooter').repeat(__.outE().inV().simplePath()).until(hasLabel('incident')).out().path().limit(50)

---

### Optional: Delete entire graph; i.e. to reset it and load data again
More at ["Empty an Amazon Neptune DB cluster using the fast reset API"](https://docs.aws.amazon.com/neptune/latest/userguide/manage-console-fast-reset.html)

In [ ]:
def delete_all_graph(conn_string):
    # Init Reset parameters
    headers = {'Content-Type': 'application/json'}
    data = {
        "action" : "initiateDatabaseReset"
    }

    # Init Database Full Reset
    response = requests.post(conn_string, data=json.dumps(data), headers=headers)

    # Get the Reset Token from API
    reset_token = response.json()['payload']['token']

    # Run reset params
    data = {
        "action" : "performDatabaseReset",
        "token" : reset_token
    }

    # Query the Neptune Loader endpoint, to collect the status
    response = requests.post(conn_string, data=json.dumps(data), headers=headers)


    # Return the status. This can be used for polling; i.e. in a Step Function workflow.
    return json.dumps(response.json(), separators=(',', ':'), indent=4)

In [ ]:
# Warning: Confirm your cluster name here, before deleting all the graph data:
PORT = 8182
SERVER_TO_TRUNCATE = 'add-your-cluster-name-here.aws-region-here.neptune.amazonaws.com'
ENDPOINT_TO_TRUNCATE = f"https://{SERVER_TO_TRUNCATE}:{PORT}/system"

### WARNING: This will DELETE all Neptune data from the cluster

In [ ]:
# WARNING: This will DELETE all Neptune data from the cluster
reset_response = delete_all_graph(conn_string=ENDPOINT_TO_TRUNCATE)
print(reset_response)

In [ ]:
%%gremlin

// Confirm this query returns nothing. This may fail, while the Database Reset is working. Execute a few times after a minute or so, and it should work.  
g.V().out().limit(10)